<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Prompts

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering   

Prompts are the fundemental building block to getting the proper responses from AI models. This module demonstrates how to use common prompt engineering techniques while using Semantic Kernel. If you've ever used ChatGPT or Microsoft Copilot, you're already familiar with prompting. Given a prompt instruction, an LLM will attempt to predict the most likely response. With Semantic Kernel, you can craft applications, services & APIs that execute prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [9]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.13.0"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.13.0

Using Azure OpenAI Service


### Step 2 - Execute a Semantic Kernel Prompt

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper decision frames
* Gather the information & intelligence in order to make a decision
* Recommend decision frameworks to make a high-quality decision
* Provide feedback to evaluate a decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks. 

In [5]:
var simplePrompt = "Provide some decision frameworks that can help improve the quality of decisions.";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simplePrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

Improving the quality of decisions can be achieved through various structured decision frameworks. Here are some widely recognized ones:

1. **SWOT Analysis**:
    - **S**trengths: Identify internal strengths that can be leveraged.
    - **W**eaknesses: Acknowledge internal weaknesses that need to be addressed.
    - **O**pportunities: Look for external opportunities that can be taken advantage of.
    - **T**hreats: Recognize external threats that could pose risks.

2. **PEST Analysis**:
    - **P**olitical: Evaluate the influence of political factors.
    - **E**conomic: Consider economic conditions and trends.
    - **S**ocial: Understand societal and cultural factors.
    - **T**echnological: Assess the impact of technological changes and advancements.

3. **Five Whys**:
    - Start with a problem statement and ask "Why?" five times to drill down to the root cause. Each answer forms the basis of the next question.
    
4. **Decision Matrix (Weighted Scoring Model)**:
    - List all

### Step 3 - Execute a Semantic Kernel Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [6]:
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simplePrompt))
{
   Console.Write(streamChunk);
}

Certainly! Decision-making frameworks can significantly enhance the quality and consistency of decisions by providing structured approaches. Here are a few well-regarded decision frameworks:

### 1. **SWOT Analysis:**
This framework helps in identifying the internal and external factors that could impact the decision.

- **Strengths:** What are the internal advantages?
- **Weaknesses:** What are the internal disadvantages?
- **Opportunities:** What external opportunities can we exploit?
- **Threats:** What external threats could harm us?

### 2. **Pros and Cons List:**
A simple yet effective method that involves listing the advantages (pros) and disadvantages (cons) of each option to weigh their merits.

### 3. **Decision Matrix Analysis (Grid Analysis):**
This tool helps to evaluate and prioritize a list of options. It involves:

- Listing options as rows.
- Setting criteria as columns.
- Scoring each option against each criterion.
- Weighting criteria based on importance.
- Calculati

### Step 4 - Execute a Semantic Kernel Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Tempature or other settings.

In [7]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.0, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simplePrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Certainly! Decision-making frameworks are structured approaches that help individuals and organizations make more informed and effective choices. Here are some widely recognized frameworks that can improve the quality of decisions:

### 1. **SWOT Analysis**
- **Strengths:** Identify internal strengths.
- **Weaknesses:** Identify internal weaknesses.
- **Opportunities:** Identify external opportunities.
- **Threats:** Identify external threats.

### 2. **PEST Analysis**
- **Political:** Consider political factors.
- **Economic:** Consider economic factors.
- **Social:** Consider social factors.
- **Technological:** Consider technological factors.

### 3. **Decision Matrix (Weighted Scoring Model)**
- List all options.
- Identify criteria important for the decision.
- Assign weights to each criterion based on importance.
- Score each option against each criterion.
- Multiply scores by weights and sum them to get a total score for each option.

### 4. **Cost-Benefit Analysis**
- List all 

### Step 5 - Execute a Semantic Kernel Prompt with a system prompt (persona)

All the best practices of prompt engineering apply when using Semantic Kernel.  
* Make the prompt more specific
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Tell the AI what to do to avoid doing something wrong
* Provide context to the AI
* Using message roles in chat completion prompts
* Give your AI words of encouragement  

One best practice is to provide a common behaviour across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. These prompts can be dynamically created with a programming language like C#.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. The decision framework responses are much more robust with decision intelligence information.

In [8]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt
var systemPromptForDecisions = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var simplePrompt = "Provide some decision frameworks that can help improve the quality of decisions.";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {simplePrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 750, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

Console.WriteLine("Prompt To Send to the Azure OpenAI Chat Completion Service...");
Console.WriteLine(simpleDecisionPromptTemplate);
Console.WriteLine(string.Empty);
Console.WriteLine("----------------------------------------");
Console.WriteLine(string.Empty);
Console.WriteLine("Response from the Azure OpenAI Chat Completion Service...");


await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simplePrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Prompt To Send to the Azure OpenAI Chat Completion Service...
System Prompt: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

Request from the user: Provide some decision frameworks that can help improve the quality of decisions.

----------------------------------------

Response from the Azure OpenAI Chat Completion Service...
Certainly! Here are several decision frameworks that can help improve the quality of decisions:

### 1. **SWOT Analysis**
**Strengths, Weaknesses, Opportunities, Threats**

- **Strengths:** Identify internal attributes that are helpful to achieving the objective.
- **Weaknesses:** Identify internal attributes that are harmful to achieving the objective.
- **Opportunities:** Identify external conditions that are helpful to achieving the objective.
- **Threats:** Identify extern